<div class='alert alert-info' style='text-align:center'><h1>Determining DICOM Image Order</h1>
- yet another MR processing notebook -</div>

#### We'll use the DICOM tag ImagePositionPatient to accurately determine the slice order of an MR series.

#### The I.P.P. tag gives us three values (X, Y and Z) that determine a slice's position along the three axis of the patient coordinate space as related to a fixed center point.

The patient coordinate space (or Reference Coordinate System in DICOM-speak) is defined as:

- X - right to left (Right -> Left)
- Y - front to back (Anterior -> Posterior)
- Z - bottom to top (Feet -> Head)

- In most cases, the InstanceNumber and SlicePosition tags can be used to determine location. But, since they're not completely required and the studies could have been post-processed .. we cannot depend on them.

In [ ]:
import os
import numpy as np
import pandas as pd
import pydicom
import matplotlib.pyplot as plt

In [ ]:
# Get a list of images in a series
def get_series_list(directory, study, series):
    series_list = []
    for subdirs, dirs, files in os.walk(directory + '/' + study + "/" + series):
        series_list = os.listdir(directory + '/' + study + '/' + series) 
    return series_list

In [ ]:
# Convert the Image Orientation Patient tag cosine values into a text string of the plane.
# This represents the plane the image is 'closest to' .. it does not explain any obliqueness
def get_image_plane(loc):

    row_x = round(loc[0])
    row_y = round(loc[1])
    row_z = round(loc[2])
    col_x = round(loc[3])
    col_y = round(loc[4])
    col_z = round(loc[5])
    if (row_x, row_y, col_x, col_y) == (1,0,0,0):
        return "Coronal"
    if (row_x, row_y, col_x, col_y) == (0,1,0,0):
        return "Sagittal"
    if (row_x, row_y, col_x, col_y) == (1,0,0,1):
        return "Axial"
    return "Unknown"

In [ ]:
directory = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train'
study = '00017'
series = 'T1wCE'
files = []

# Get a list of images for this study/series
series_list = get_series_list(directory, study, series)

if len(series_list) > 0:
    for f in series_list:
        
        # Read the image and get it's orientation and position tags
        image = pydicom.dcmread(f'{directory}/{study}/{series}/{f}')
        plane = get_image_plane(image[0x0020,0x0037])
        
        # Make a list
        files.append([f, plane, float(image[0x0020,0x0032].value[0]), float(image[0x0020,0x0032].value[1]), float(image[0x0020,0x0032].value[2])])

In [ ]:
# Convert the list of files and position coords to a dataframe
df = pd.DataFrame(data=files, columns=('image','plane','iop_x','iop_y','iop_z'))
df.head(10)

#### Now, we have an unordered list of images and their X, Y and Z positions.
- Notice the X and Y positions aren't changing at all in this series. That's because these slices are perfectly orthogonal to the patient's body coordinate system.
- But, since the plane is axial, we want to sort on the Z coord (Feet->Head)

In [ ]:
# Sort the dataframe on the iop_z column
df = df.sort_values(by=['iop_z'])
df.head(10)

### Now our dataframe is in the proper order.
- In this example, the filenames match the image order .. as most will. However, it's not safe to assume it will always be the case.
- Let's try another study.

In [ ]:
study = '00148'
series = 'T1wCE'
files = []

# Get a list of images for this study/series
series_list = get_series_list(directory, study, series)

if len(series_list) > 0:
    for f in series_list:
        
        # Read the image and get it's orientation and position tags
        image = pydicom.dcmread(f'{directory}/{study}/{series}/{f}')
        plane = get_image_plane(image[0x0020,0x0037])
        
        # Make a list
        files.append([f, plane, float(image[0x0020,0x0032].value[0]), float(image[0x0020,0x0032].value[1]), float(image[0x0020,0x0032].value[2])])

In [ ]:
# Convert the list of files and position coords to a dataframe
df = pd.DataFrame(data=files, columns=('image','plane','iop_x','iop_y','iop_z'))
df.head(10)

### In this case, it's not easy to determine which position we're interested in because *all* the positions seem to change ..
- This is because these images are not truely orthogonal. They are obliqued slightly to accomodate the patient's rotation.
- In any case, we know these are sagittal slices, so we are interested in the X coordinate. We'll sort on it.

In [ ]:
# Sort the dataframe on the iop_x column
df = df.sort_values(by=['iop_x'])
df


### And now we can see the images are ordered from Right->Left, with the highest filename being the image on the patient's extreme right.
- This technique can be used to arrange DICOM series into the proper order and to determine the scan direction.

#### Some of my other MR notebooks
- Tumor Object Detection -> https://www.kaggle.com/davidbroberts/brain-tumor-object-detection
- Determining MR Slice Orientation -> https://www.kaggle.com/davidbroberts/determining-mr-slice-orientation
- Determining MR image planes -> https://www.kaggle.com/davidbroberts/determining-mr-image-planes
- Reference Lines on MR images -> https://www.kaggle.com/davidbroberts/mr-reference-lines
- Manual VOI LUT on MR images -> https://www.kaggle.com/davidbroberts/manual-voi-lut-on-mr-images
- Export DICOM Images by Plane -> https://www.kaggle.com/davidbroberts/export-dicom-series-by-plane/